In [1]:
import gzip
import re

In [2]:
with gzip.open("../data/poczatki_wikipediowe.txt.gz", "rt") as f:
    file_content = f.read()

In [3]:
d = {}
counter = 0
for part in file_content.removeprefix("### ").split("\n### "):
    splited = part.strip().split("\n", maxsplit=1)
    if len(splited) == 2:
        title, content = splited
        if title in d:
            counter += 1
        d[title] = content

In [4]:
def peek(pattern, content):
    idx = file_content.find(pattern)
    return content[idx - 1000: idx + 1000]

In [5]:
peek("Arabska stolica perfum", file_content)

"wybrany królem zachodniofrankijskim. Jego potomkowie walczyli w Galii z Karolingami o władzę. W końcu Hugo\n\n### Leszczyc\n* Leszczyc (herb szlachecki)\n\n### Bróg (herb szlachecki)\n\n\n### Brożek (herb szlachecki)\n\n\n### Laska (herb szlachecki)\n\n\n### Laski (herb szlachecki)\n\n\n### Wyszowie (herb szlachecki)\n\n\n### Diogo Freitas do Amaral\n'''Diogo Pinto de Freitas do Amaral''' (ur. 21 lipca 1941 w Povoa de Varzim) – prawnik i polityk portugalski, współzałożyciel i wieloletni przewodniczący Centrum Demokratyczno-Społecznego, tymczasowy szef rządu od 4 grudnia 1980 do 9 stycznia 1981. W 1980 i w latach 2005–2006 minister spraw zagranicznych. = = Życiorys = = W 1963 obronił licencjat, a cztery lata później\n\n### Salala\n350px '''Salala''' (arab. '''صلالة''') - miasto na zachodzie Omanu, nad Morzem Arabskim. Obok miasta znajduje się ważny dla Omanu szlak komunikacyjny: droga Maskat - Salala. Ludność: 190 tys. (2008 rok) - czwarte co do wielkości miasto kraju. = = Turystyka = 

In [6]:
sum(v.startswith("'''") for v in d.values())

561805

In [11]:
skrot_pattern = re.compile(r"'''([\w\s]+)''' \(w skrócie '''([\w\s]+)'''\)")
re.findall(skrot_pattern, file_content)[:5]

[('Quake C', 'QC'),
 ('Senatus Populusque Romanus', 'SPQR'),
 ('General Motors Corporation', 'GM'),
 ('Państwowe Gospodarstwo Rolne', 'PGR'),
 ('Generalny Inspektor Ochrony Danych Osobowych', 'GIODO')]

In [145]:
base_pattern = re.compile(r"'''([\w\s]+)'''(?:(?:,|(?:,? lub)) '''([\w\s]+)''')*")
definition_pattern = re.compile(r"'''(?<!,)([\w\s]+)'''")

def extract_synonyms(title, content):
    if content.startswith("* "):
        # enumeration
        return set()
    elif m := base_pattern.match(content):
        synonyms = set(definition_pattern.findall(m.group()))
        for skrot_set in map(set, skrot_pattern.findall(content)):
            if not skrot_set.isdisjoint(synonyms):
                synonyms.update(skrot_set)
        return synonyms
    else:
        return set()

In [146]:
m = base_pattern.match("'''Wizygoci''', '''Goci zachodni''' lub '''Terwingowie''' (dosł. leśni ludzie ) – lud germańs")
print(definition_pattern.findall(m.group()))
print(m.group())

['Wizygoci', 'Goci zachodni', 'Terwingowie']
'''Wizygoci''', '''Goci zachodni''' lub '''Terwingowie'''


In [159]:
translation_pattern = r" ?\(\w+\. ''[\w\s]+''\)"

In [ ]:
# counter = 0

# for k, v in d.items():
#     if v.startswith("* "):
#         # enumeration
#         pass
#     if (m1:= another_pattern.match(v)) and (m2:= basic_pattern.match(v)):
#         if m1.group() != m2.group():
#             if len(basic_extract_pattern.findall(m2.group())) > 1:
#                 print(v)
#                 counter += 1
#                 if counter > 20:
#                     break

In [158]:
re.sub(
    r" ?\(\w+\. ''[\w\s]+''\)",
    "",
    "'''Emotikon''' lub '''emotikona''' (ang. ''emoticon''), '''uśmieszek''' (''smile'', ''smiley''), '''buźka''' – złożony ze zn"
)

"'''Emotikon''' lub '''emotikona''', '''uśmieszek''' (''smile'', ''smiley''), '''buźka''' – złożony ze zn"

In [147]:
m = base_pattern.match("'''Emotikon''' lub '''emotikona''' (ang. ''emoticon''), '''uśmieszek''' (''smile'', ''smiley''), '''buźka''' – złożony ze zn")
print(definition_pattern.findall(m.group()))
re.
print(m.group())

['Emotikon', 'emotikona']
'''Emotikon''' lub '''emotikona'''


In [19]:
another_pattern = re.compile(r"'''([\w\s]+)'''(?:, '''([\w\s]+)''')*(?: lub '''([\w\s]+)''')*")

def extract_synonyms(title, content):
    if content.startswith("* "):
        # enumeration
        return set()
    elif m := another_pattern.match(content):
        return set(basic_extract_pattern.findall(m.group()))
    else:
        return set()

In [13]:
with open("out", "wt") as f:
    for k, v in d.items():
        if v.startswith("* "):
            # enumeration
            continue

        synonyms = extract_synonyms(k, v)
        if len(synonyms) > 1:
            f.write(" # ".join(synonyms) + "\n")

In [141]:
useful = [
    "Wizygoci", "Emotikon", "Reakcja termojądrowa",
    "Zabytki Głogowa", "Powiat wasylkowski", "Psianka podłużna",
    "Salala", "Komórka blastyczna", "Kasa Rolniczego Ubezpieczenia Społecznego",
]
for title in useful:
    print(title, d[title], "Extracted: " + str(extract_synonyms(title, d[title])), sep="\n", end="\n\n")

Wizygoci
'''Wizygoci''', '''Goci zachodni''' lub '''Terwingowie''' (dosł. leśni ludzie ) – lud germański, odłam Gotów. W IV wieku przyjęli arianizm (dzięki Biblii przetłumaczonej przez Wulfilę na język gocki). = = Wczesne dzieje = = Pierwsze przekazy traktujące Wizygotów jako osobny lud odwołują się do roku 268, w którym najechali oni cesarstwo rzymskie i opanowali Półwysep
Extracted: {'Terwingowie', 'Goci zachodni', 'Wizygoci'}

Emotikon
'''Emotikon''' lub '''emotikona''' (ang. ''emoticon''), '''uśmieszek''' (''smile'', ''smiley''), '''buźka''' – złożony ze znaków tekstowych (ASCII) wyraz nastroju, używany przez użytkowników Internetu. Najczęściej przedstawia symboliczny ludzki grymas twarzy, obrócony o 90° w kierunku przeciwnym do wskazówek zegara, w licznych wariantach. Niektóre programy jak np.
Extracted: {'Emotikon', 'emotikona'}

Reakcja termojądrowa
'''Reakcja termojądrowa''', '''synteza jądrowa''' lub '''fuzja jądrowa''' – '''Reakcja termojądrowa''', '''synteza jądrowa''' lub '